In [1]:
import argparse
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

#parser = argparse.ArgumentParser(description="Calculates accuracy of centroid models.")
#parser.add_argument('label_input', help='CSV file containing true centroid values. This can be made from converting research XML files from LABEL_PARTY folder using research_centroids.py')
#parser.add_argument('test_input', help='CSV file to test, formatted with labels /"#, frame, x, y/"')
#args = parser.parse_args()
#label_df = pd.read_csv(args.label_input)
#test_df = pd.read_csv(args.test_input)

LABEL_DF = pd.read_csv('./../research_data/input_mp4.csv') #Ground Truth Data
TEST_DF = pd.read_csv('./../data/4_22_2020_mask.mp4_blobdetector.csv') #Points we are testing
#delete above two lines and replace with commented code when finished

#sorting the data by the video frame, 1st frame, 2nd, 3rd, etc.
label_df = LABEL_DF.sort_values('frame')
test_df = TEST_DF.sort_values('frame')
#cleaning data, removing unnecessary columns & relabeling
all_label = label_df.copy()
del test_df['Unnamed: 0']
del label_df['velocity']
del label_df['baboon id']
label_df = label_df.rename(columns={"centroid_x": "x", "centroid_y": "y"})
print(label_df)
print(all_label)
print(label_df.shape)
print(all_label.shape)
print(test_df)

        frame         x         y
39508       0  1622.100  1243.030
42082       0  1532.255  1374.540
94083       0  2269.810  1784.140
55099       0  2281.960   921.275
113565      0  1625.140  1224.935
...       ...       ...       ...
19520    1798  1632.690  1233.320
79690    1798  2485.320   742.845
20232    1798  1807.100  1198.130
14349    1798  2176.115  1328.070
124893   1798  2810.710  1636.925

[124894 rows x 3 columns]
        baboon id  frame  centroid_x  centroid_y   velocity
39508          50      0    1622.100    1243.030        NaN
42082          52      0    1532.255    1374.540        NaN
94083          90      0    2269.810    1784.140        NaN
55099          62      0    2281.960     921.275        NaN
113565        104      0    1625.140    1224.935        NaN
...           ...    ...         ...         ...        ...
19520          18   1798    1632.690    1233.320   1.948050
79690          81   1798    2485.320     742.845   1.613934
20232          19   1798 

In [ ]:

all_frames = label_df['frame'].unique() #list of frames where there are baboons

#Initialize k-d tree
nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree')

#Cycles through every frame that we have Ground Truth data in
found_baboons = pd.DataFrame()
print(found_baboons)
alldist = ([[0]])
for frame in all_frames:
    #print(frame)
    fill_set = label_df.loc[label_df['frame'] == frame] #grabs all baboons identified in that frame from Ground Truth data
    velocity_set = all_label.loc[label_df['frame'] == frame]
    #print(fill_set)
    #print(velocity_set)
    del fill_set['frame']
    nbrs.fit(fill_set.to_numpy()) #fill k-d tree with all Ground Truth positions of baboons in that frame
    find_set = test_df.loc[test_df['frame'] == frame] #grabs baboon positions for this frame from test data
    #print(find_set)
    del find_set['frame']
    find_num = find_set.to_numpy()#convert to npy / req. to pass to kneighbors
    if find_num.size == 0: #edge case
        continue
    distances, indices = nbrs.kneighbors(find_num)
    #print(indices.flatten())
    add_df = velocity_set.iloc[indices.flatten()]
    add_df['Distance'] = distances.flatten()
    print(add_df)
    found_baboons = found_baboons.append(add_df)
    #print("found_baboons")
    #print(found_baboons)
    alldist = np.concatenate((alldist, distances))
print("Finished collecting distances.")
#print(alldist)
print(found_baboons)


Empty DataFrame
Columns: []
Index: []
       baboon id  frame  centroid_x  centroid_y  velocity   Distance
68342         74      0    2845.110    1620.055       NaN   3.140596
43881         53      0    1433.945     687.880       NaN   7.903235
63483         69      0    2904.545    1196.445       NaN   4.795803
36049         46      0    1543.260    1292.760       NaN   3.415744
64512         70      0    2969.785    1285.150       NaN  11.300205
34250         45      0    1431.270    1254.720       NaN   3.537446
70141         75      0    2328.095    1397.995       NaN   1.278372
65801         71      0    2740.460    1445.955       NaN  11.656734
65896         72      0    2804.565    1519.990       NaN  12.798909
67344         73      0    2959.100    1510.195       NaN  15.688107
42082         52      0    1532.255    1374.540       NaN   6.766397
       baboon id  frame  centroid_x  centroid_y   velocity   Distance
64513         70      1    2971.045    1286.285  50.823898  10.1

        baboon id  frame  centroid_x  centroid_y    velocity   Distance
38015          48     11    1617.105    1304.215   21.915530   3.413626
63494          69     11    2924.245    1215.635   63.861422   7.975857
34261          45     11    1423.000    1254.545   15.311855  10.867165
64523          70     11    2983.590    1297.340   45.055786   3.722577
70152          75     11    2317.785    1389.370   41.728584   8.423744
43892          53     11    1441.705     680.530   46.392553   7.261008
70379          76     11    2342.735    1436.040    7.170914   4.496326
65812          71     11    2767.430    1462.145  125.643037   4.400869
67355          73     11    2969.710    1525.965   37.377889   1.233012
65907          72     11    2824.660    1536.985   64.688702   6.176980
105506         97     11    2937.730    1841.135   12.230000   3.386986
105673         98     11    2423.630    1409.200    4.908588   1.142595
        baboon id  frame  centroid_x  centroid_y    velocity  Di

/Users/christiewolters/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/christiewolters/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/christiewolters/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 


        baboon id  frame  centroid_x  centroid_y    velocity   Distance
43896          53     15    1446.485     676.590   46.507849   3.798968
59562          65     15    2495.620    1199.490   21.419641   1.477372
63498          69     15    2931.235    1220.555   64.106383   8.649236
38019          48     15    1619.940    1303.505   21.915530   2.798628
70156          75     15    2312.915    1385.540   51.020349  13.013343
70383          76     15    2342.565    1437.390   10.260075   4.049552
65816          71     15    2781.795    1470.810  125.565716   7.854910
65816          71     15    2781.795    1470.810  125.565716  19.445564
67359          73     15    2972.170    1530.290   37.377889   0.588735
65911          72     15    2831.460    1542.295   64.806790   1.704478
105677         98     15    2424.175    1408.855    4.862627   3.568313
        baboon id  frame  centroid_x  centroid_y    velocity   Distance
70157          75     16    2311.575    1384.490   51.020349  1

        baboon id  frame  centroid_x  centroid_y    velocity   Distance
59574          65     27    2506.285    1201.580   31.470284  12.767500
63510          69     27    2945.635    1232.375   34.331024   9.517027
70168          75     27    2294.570    1363.465  155.211329  12.818529
38031          48     27    1627.945    1303.860   19.924979   2.015757
70395          76     27    2347.085    1442.915   27.693480   1.293429
65828          71     27    2805.990    1496.825   70.292269   9.296745
65923          72     27    2855.920    1552.910   67.489253  18.231347
105689         98     27    2437.095    1399.570   67.744300  10.152383
        baboon id  frame  centroid_x  centroid_y    velocity   Distance
65924          72     28    2857.615    1554.410   67.834399  17.359046
65829          71     28    2806.880    1498.995   70.292269   8.991580
70396          76     28    2347.945    1443.235   27.500639   1.015338
105690         98     28    2438.975    1398.315   67.744300   9

...

/Users/christiewolters/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/christiewolters/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/christiewolters/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

        baboon id  frame  centroid_x  centroid_y   velocity   Distance
46297          55    270    1730.400     893.435  67.911154   7.752240
58018          64    270    2231.000     928.765  75.463127   4.584134
113835        104    270    1670.365    1229.620  33.043539   4.801140
59817          65    270    2632.665    1281.405  49.360736   3.560153
34179          44    270    1604.790    1243.960  19.857811   6.209560
109886        101    270    2128.780    1303.895  17.022986   6.166877
2970            3    270    2784.240    1441.430  43.201784   9.256216
4688            4    270    2824.740    1521.475  96.167047  11.421231
31646          41    270    2948.600    1888.205   7.250325  10.001427
41577          51    270    1631.330    1302.360  15.506398   2.959590
        baboon id  frame  centroid_x  centroid_y   velocity   Distance
46298          55    271    1728.260     892.690  67.911154   6.466475
58019          64    271    2229.860     931.005  75.326720   5.411900
113836

        baboon id  frame  centroid_x  centroid_y    velocity   Distance
4700            4    282    2850.840    1551.305   99.490018  20.978368
2982            3    282    2797.385    1470.580   83.664088   9.390392
59829          65    282    2631.955    1292.870   21.444786   4.050149
113847        104    282    1679.195    1232.255   21.160178   4.712238
60588          66    282    2385.505    1211.670    2.763097   8.037789
4785            5    282    1818.030    1157.330   13.181690   2.315480
58030          64    282    2214.575     958.790  117.587573   8.274660
        baboon id  frame  centroid_x  centroid_y    velocity   Distance
60589          66    283    2385.415    1211.640    2.843204   8.456922
4786            5    283    1818.310    1157.660   12.970468   2.037681
113848        104    283    1679.870    1232.440   20.975789   4.601192
58031          64    283    2212.675     962.225  117.645993   7.752229
59830          65    283    2631.630    1293.520   21.779861  14

/Users/christiewolters/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/christiewolters/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/christiewolters/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

        baboon id  frame  centroid_x  centroid_y    velocity   Distance
4704            4    286    2860.480    1557.255   81.711269  19.493422
2986            3    286    2805.700    1484.515  134.525329   6.730812
59833          65    286    2632.385    1298.640   28.911731   6.289119
113851        104    286    1682.795    1233.940   39.014563   3.049427
34195          44    286    1614.920    1232.465   18.886451  13.178066
60592          66    286    2385.135    1211.570    2.985740   6.315950
58034          64    286    2208.420     973.420  119.701733   8.649586
        baboon id  frame  centroid_x  centroid_y    velocity   Distance
58035          64    287    2207.000     977.155  119.754904   7.745024
60593          66    287    2385.045    1211.550    2.763097   6.223214
59834          65    287    2632.745    1299.540   29.050810   6.613776
113852        104    287    1683.920    1234.600   39.090176   2.350693
2987            3    287    2808.105    1488.305  134.525329   7

In [ ]:
found_baboons = found_baboons.sort_values('velocity')
found_baboons = found_baboons.drop_duplicates()
found_baboons = found_baboons.dropna()
#found_baboons = found_baboons[found_baboons['Distance'] < 25]
pd.options.display.max_rows
pd.set_option('display.max_rows', None)
print(found_baboons)

In [ ]:
#Plot the Euclidian distance data on a histogram
def map_histo(data, range_min, range_max, bins, rwidth = 0.2):
    range = (range_min, range_max)
    print("Range:", range)
    plt.hist(data, bins, range, color = 'green', histtype = 'bar', rwidth = rwidth)
    plt.xlabel('Velocity') 
    plt.ylabel('No. of baboons') 
    plt.title('Speed of Baboons') 
    plt.show() 

In [ ]:
map_histo(found_baboons.velocity, 0, 150, 49)

In [ ]:
map_histo(LABEL_DF.velocity.dropna(), 19.5, 150, 49)

In [ ]:
#The number of baboon values that lie between 0-0.99 1-1.99 2-2.99 etc etc
#First is in the found baboons
split_mylist = pd.cut(found_baboons.velocity, list(range(0, 200))).value_counts(sort = False)
print(split_mylist.tolist())
#second is in the LABEL PARTY (after NAN removed)
split_labellist = pd.cut(LABEL_DF.velocity.dropna(), list(range(0, 200))).value_counts(sort = False)
print(split_labellist.tolist())
#finally determine how well it is picking up these velocities by dividing
res = zip(split_mylist, split_labellist)
list_percent = []
for i, j in res:
    if j == 0:
        list_percent.append(0)
    else:
        list_percent.append(i/j)
list_percent = [i * 100 for i in list_percent]
#res = [i / j for i, j in zip(split_mylist, split_labellist)]
print(list_percent)

In [ ]:
#plot the percentages
plt.plot(list(range(0,80)), list_percent[:80])
plt.ylabel('Percent Accurate')
plt.xlabel('velocity')
plt.show()

In [ ]:
moving_df = LABEL_DF.copy()
print(moving_df.shape)
print(found_baboons.shape)
moving_df = moving_df.dropna()
moving_df = moving_df[moving_df['velocity'] > 5]
these_baboons = found_baboons[found_baboons['velocity'] > 5]
print(moving_df.shape)
print(these_baboons.shape)

In [ ]:
map_histo(alldist, int(min(alldist)), int(max(alldist)), 100)

In [ ]:
map_histo(alldist, 0, 250, 126)

In [ ]:
map_histo(alldist, 0, 50, 51)

In [ ]:
map_histo(alldist, 0, 30, 31, 0.5)

In [ ]:
#Use Z-score to identify outliers
#Z score = (Observation — Mean)/Standard Deviation


def remove_outliers(data, standard_deviations=3):
    keepers=[]
    threshold=standard_deviations #number of standard deviations above mean
    mean = np.mean(data)
    print("REMOVING OUTLIERS")
    print("Mean: ", mean)
    std =np.std(data)
    print("Standard Deviation: ", std)
    
    for y in data:
        z_score= (y - mean)/std
        if np.abs(z_score) <= threshold:
            keepers.append(y)
    print("Before removal: ", len(data))
    print("After removal: ", len(keepers))
    print( (1-(len(keepers)/len(data))) * 100, "% lost in removal.")
    return keepers
clean_datapoints = remove_outliers(alldist[:,0].tolist(), 1)

In [ ]:
clean_datapoints2 = remove_outliers(clean_datapoints)

In [ ]:
print( (1-(len(clean_datapoints2)/alldist.shape[0])) * 100, "% lost in total.")

In [ ]:
print("Trimming to distances < 25")
trim = alldist[alldist < 25]
print((len(trim)/len(alldist))*100, "%")

In [ ]:
_ = remove_outliers(trim)

In [ ]:
#How many baboons does it predict out of all frames and all baboons?
print("Total number of instances of baboons in Ground Truth data(Every baboon identified in every frame of the video, summed):")
print(label_df.shape[0])
print("Total number of instances of baboons identified by data set to 25 pixels:")
print(len(trim))
print(len(trim)/label_df.shape[0]*100)